In [10]:
from transformers import GPT2Tokenizer

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a padding token
# tokenizer.pad_token = tokenizer.eos_token  # Or use a custom token like "<|pad|>"

# If using a custom padding token:
# tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
# tokenizer.pad_token = '<|pad|>'

In [11]:
tokenizer.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "pad_token": "<pad>",
    "unk_token": "<unk>"
})

# Check after adding
print(tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token, tokenizer.unk_token)
# Output: ('<s>', '</s>')

<s> </s> <pad> <unk>


In [14]:
tokenizer.decode(tokenizer.pad_token_id)

'<pad>'

In [6]:
tokenizer.encode("Hello bro hi hi", add_special_tokens=True)

[15496, 1379, 23105, 23105]